In [1]:
import camelot
import pandas as pd
import os
from dotenv import load_dotenv
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

load_dotenv()
openai.api_key = os.getenv("OPENAIAPIKEY")

# Scrape Tables From PDF

In [2]:
pages = ''
for i in range(3, 43):
    if i < 42:
        pages += str(i)+', '
    else:
        pages += str(i)

In [3]:
tables = camelot.read_pdf('pix_participants.pdf', pages=pages, strip_text='.\n')
tables.n

40

In [4]:
df_list = []
for i in range(tables.n):
    df_list.append(tables[i].df)
    
df = pd.concat(df_list)

In [5]:
df=df.drop(0, axis=1)

#### Output Raw df to CSV

In [6]:
#df.columns = df.iloc[0]
#df.drop(0, axis=1)
#df.to_csv('raw_pix_participants.csv')

# Use Chat GPT to Translate Fields to English

https://towardsdatascience.com/translate-with-chatgpt-f85609996a7f

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

for i in range(len(df)):
    
    # Get Values from current row to translate
    columnsToTranslate = [1, 3, 4, 5, 6, 7, 8]
    for c in columnsToTranslate:
        textString = df.iloc[i][c]
        
        # Create the prompt
        prompt = f'Please provide the English translation for these words: {textString}'
    
        # Translate values using exponential backoff to prevent rate limit errors
        response = completion_with_backoff(
          model="text-davinci-003",
          prompt=prompt,
          max_tokens=2000,
          top_p=0.2,
          frequency_penalty=1.0,
          presence_penalty=1.0
        )

        resultString = response.choices[0].text.replace('\n', '')
    
        # Set new values in df
        df.at[i, c] = resultString